In [ ]:
import os
import json
import zipfile
from tqdm import tqdm
from collections import Counter

# ===== Solution Validation =====
def check_solution(code_str, task_data):
    """Check if a code solution works for all examples in the task"""
    try:
        local_vars = {}
        exec(code_str, {}, local_vars)
        f = local_vars['p']
        
        # Test on all available examples
        examples = task_data['train'] + task_data['test'] + task_data['arc-gen']
        for ex in examples:
            result = f(ex['input'])
            if result != ex['output']:
                return False
        return True
    except:
        return False

def code_length(code):
    """Calculate code length in bytes"""
    return len(code.encode('utf-8'))

# ===== Grid Analysis Utilities =====
def get_colors(grid):
    """Get all unique colors in a grid"""
    colors = set()
    for row in grid:
        colors.update(row)
    return colors

def get_dimensions(grid):
    """Get grid dimensions (height, width)"""
    return len(grid), len(grid[0]) if grid else 0

def count_colors(grid):
    """Count occurrences of each color"""
    counter = Counter()
    for row in grid:
        counter.update(row)
    return counter

def most_common_color(grid):
    """Find the most frequent color in grid"""
    counts = count_colors(grid)
    return counts.most_common(1)[0][0] if counts else 0

def analyze_transformation(inp, out):
    """Analyze what transformation was applied"""
    inp_dims = get_dimensions(inp)
    out_dims = get_dimensions(out)
    inp_colors = get_colors(inp)
    out_colors = get_colors(out)
    
    return {
        'same_size': inp_dims == out_dims,
        'inp_dims': inp_dims,
        'out_dims': out_dims,
        'inp_colors': inp_colors,
        'out_colors': out_colors,
        'color_change': inp_colors != out_colors
    }

# ===== Basic Transformation Generators =====
def identity():
    """Return input unchanged"""
    return "def p(g):return g"

def flip_vertical():
    """Flip grid vertically"""
    return "def p(g):return g[::-1]"

def flip_horizontal():
    """Flip grid horizontally"""
    return "def p(g):return[r[::-1]for r in g]"

def rotate_90():
    """Rotate grid 90 degrees clockwise"""
    return "def p(g):return list(map(list,zip(*g[::-1])))"

def rotate_180():
    """Rotate grid 180 degrees"""
    return "def p(g):return[r[::-1]for r in g[::-1]]"

def rotate_270():
    """Rotate grid 270 degrees clockwise"""
    return "def p(g):return list(map(list,zip(*g)))[::-1]"

def transpose():
    """Transpose grid (swap rows and columns)"""
    return "def p(g):return list(map(list,zip(*g)))"

def fill_zeros():
    """Fill entire grid with zeros"""
    return "def p(g):return[[0]*len(g[0])for _ in g]"

def fill_ones():
    """Fill entire grid with ones"""
    return "def p(g):return[[1]*len(g[0])for _ in g]"

# ===== Color Replacement Generators =====
def replace_color_simple(from_col, to_col):
    """Replace one color with another"""
    return f"def p(g):return[[{to_col}if x=={from_col}else x for x in r]for r in g]"

def replace_nonzero_with_color(color):
    """Replace all non-zero values with specified color"""
    return f"def p(g):return[[{color}if x!=0 else 0 for x in r]for r in g]"

def replace_zero_with_color(color):
    """Replace all zero values with specified color"""
    return f"def p(g):return[[{color}if x==0 else x for x in r]for r in g]"

def swap_two_colors(c1, c2):
    """Swap two specific colors"""
    return f"def p(g):return[[{c2}if x=={c1}else {c1}if x=={c2}else x for x in r]for r in g]"

def map_to_binary():
    """Map non-zero to 1, zero to 0"""
    return "def p(g):return[[1if x!=0 else 0 for x in r]for r in g]"

def invert_binary():
    """Invert binary grid (0->1, 1->0)"""
    return "def p(g):return[[0if x==1 else 1if x==0 else x for x in r]for r in g]"

# ===== Pattern-Based Generators =====
def extract_first_value():
    """Use first cell value to fill entire grid"""
    return "def p(g):v=g[0][0];return[[v]*len(g[0])for _ in g]"

def extract_most_common():
    """Fill grid with most common color"""
    return "def p(g):c=max(set(x for r in g for x in r),key=lambda x:sum(r.count(x)for r in g));return[[c]*len(g[0])for _ in g]"

def keep_only_color(color):
    """Keep only specified color, replace others with 0"""
    return f"def p(g):return[[x if x=={color}else 0 for x in r]for r in g]"

# ===== Advanced Pattern Generators =====
def crop_to_half():
    """Crop to upper-left quarter"""
    return "def p(g):h,w=len(g)//2,len(g[0])//2;return[r[:w]for r in g[:h]]"

def border_only():
    """Keep only border pixels"""
    return "def p(g):h,w=len(g),len(g[0]);return[[g[i][j]if i in(0,h-1)or j in(0,w-1)else 0 for j in range(w)]for i in range(h)]"

def remove_border():
    """Remove border, keep interior"""
    return "def p(g):return[r[1:-1]for r in g[1:-1]]if len(g)>2 and len(g[0])>2 else[[0]]"

# ===== Solution Generation Pipeline =====
def generate_color_solutions(task_data):
    """Generate color-based solutions from task analysis"""
    solutions = []
    
    if not task_data['train']:
        return solutions
    
    # Analyze first training example
    first_example = task_data['train'][0]
    inp, out = first_example['input'], first_example['output']
    
    # Get color information
    inp_colors = list(get_colors(inp))
    out_colors = list(get_colors(out))
    
    # Simple color replacements
    if inp and inp[0]:
        first_color = inp[0][0]
        for target in range(10):
            solutions.append(replace_color_simple(first_color, target))
    
    # Replace each input color with each possible output color
    for inp_c in inp_colors:
        for out_c in out_colors:
            if inp_c != out_c:
                solutions.append(replace_color_simple(inp_c, out_c))
    
    # Try common color operations
    for color in range(10):
        solutions.extend([
            replace_nonzero_with_color(color),
            replace_zero_with_color(color),
            keep_only_color(color)
        ])
    
    # Color swaps between common colors
    common_colors = [0, 1, 2, 3, 4, 5]
    for i, c1 in enumerate(common_colors):
        for c2 in common_colors[i+1:]:
            solutions.append(swap_two_colors(c1, c2))
    
    return solutions

def generate_geometric_solutions():
    """Generate geometric transformation solutions"""
    return [
        identity(),
        flip_vertical(),
        flip_horizontal(),
        rotate_90(),
        rotate_180(),
        rotate_270(),
        transpose(),
    ]

def generate_fill_solutions():
    """Generate solutions that fill the entire grid"""
    solutions = []
    
    # Basic fills
    solutions.extend([
        fill_zeros(),
        fill_ones(),
        extract_first_value(),
        extract_most_common(),
        map_to_binary(),
        invert_binary()
    ])
    
    # Fill with specific colors
    for color in range(2, 10):
        solutions.append(f"def p(g):return[[{color}]*len(g[0])for _ in g]")
    
    return solutions

def generate_structural_solutions():
    """Generate solutions that change grid structure"""
    return [
        crop_to_half(),
        border_only(),
        remove_border(),
    ]

def solve_task(task_data):
    """Main solving function for a single task"""
    all_solutions = []
    
    # Generate different types of solutions
    all_solutions.extend(generate_geometric_solutions())
    all_solutions.extend(generate_color_solutions(task_data))
    all_solutions.extend(generate_fill_solutions())
    all_solutions.extend(generate_structural_solutions())
    
    # Test all solutions
    valid_solutions = []
    for solution in all_solutions:
        if check_solution(solution, task_data):
            length = code_length(solution)
            valid_solutions.append((length, solution))
    
    # Return best solution (shortest code)
    if valid_solutions:
        valid_solutions.sort()
        best_length, best_code = valid_solutions[0]
        score = max(1, 2500 - best_length)
        return best_code, score, True
    else:
        # Fallback to identity
        return identity(), 0.001, False

# ===== Main Execution =====
def main():
    """Main execution function"""
    input_tasks_dir = "/kaggle/input/google-code-golf-2025"
    output_dir = "/kaggle/working/submission"
    os.makedirs(output_dir, exist_ok=True)
    
    total_solved = 0
    total_score = 0
    solved_tasks = []
    
    # Process all tasks
    for task_num in tqdm(range(1, 401), desc="Processing tasks"):
        task_id = f"{task_num:03d}"
        task_file = os.path.join(input_tasks_dir, f"task{task_id}.json")
        
        try:
            with open(task_file, 'r') as f:
                task_data = json.load(f)
            
            # Solve the task
            best_code, score, solved = solve_task(task_data)
            
            if solved:
                total_solved += 1
                solved_tasks.append(task_num)
            
            total_score += score
            
            # Write solution to file
            output_file = os.path.join(output_dir, f"task{task_id}.py")
            with open(output_file, 'w') as f:
                f.write(best_code)
                
        except Exception as e:
            print(f"Error processing task {task_id}: {e}")
            # Write identity as fallback
            output_file = os.path.join(output_dir, f"task{task_id}.py")
            with open(output_file, 'w') as f:
                f.write(identity())
            total_score += 0.001
    
    # Create submission zip
    with zipfile.ZipFile("/kaggle/working/submission.zip", "w") as archive:
        for i in range(1, 401):
            task_file = os.path.join(output_dir, f"task{i:03d}.py")
            if os.path.exists(task_file):
                archive.write(task_file, arcname=f"task{i:03d}.py")
    
    # Print results
    print(f"Tasks solved: {total_solved}/400")
    print(f"Success rate: {total_solved/400*100:.1f}%")
    print(f"Total score: {total_score:.3f}")
    print(f"Average score per task: {total_score/400:.3f}")
    print(f"Solved task numbers: {sorted(solved_tasks)[:20]}{'...' if len(solved_tasks) > 20 else ''}")
    print("Submission file 'submission.zip' created successfully!")

if __name__ == "__main__":
    main()